lr不能做回归，简单看下xgboost生成特征

In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2

import pandas as pd
from sklearn.model_selection import train_test_split

/home/baogong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import sys
sys.path.append('./util/')
from utils import load_data

users, movies, ratings = load_data()

In [5]:
data1 = pd.merge(ratings.drop(columns = ['timestamp'],axis = 1), movies, how = 'left', on = 'movieid')
data = pd.merge(data1, users, how = 'left', on = 'userid')

X = data.drop(columns = ['userid', 'movieid', 'genres', 'title', 'rating'])
Y = data['rating'].values

from sklearn import preprocessing
X_norm = preprocessing.scale(X)

train_x, test_x, train_y, test_y = train_test_split(X_norm, Y)

/home/baogong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by the scale function.
  


In [9]:
from xgboost.sklearn import XGBClassifier
import numpy as np
m = XGBClassifier(random_state=1, max_depth=5, min_child_weight=1, learning_rate=0.1, subsample=0.85,
                      colsample_bytree=0.85, n_estimators=400, n_jobs=12, objective="binary:logistic")
m.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=12, nthread=None, objective='multi:softprob', random_state=1,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.85)

In [11]:
xgb_train_x = m.apply(train_x)
xgb_test_x = m.apply(test_x)

xgb_max = float(xgb_train_x.max())
train_x_new = np.concatenate((train_x, xgb_train_x/xgb_max),axis = 1)
test_x_new = np.concatenate((test_x, xgb_test_x/xgb_max),axis = 1)

In [12]:
input_shape = train_x_new.shape[1]
learning_rate = 0.01

linear_input = tf.keras.layers.Input(shape = (input_shape,), name = "linear")
outputs = tf.keras.layers.Dense(1, name = "outputs")(linear_input)

model = tf.keras.Model(inputs = [linear_input], outputs = [outputs])

optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
model.compile(loss='mean_squared_error',
            optimizer=optimizer,
            metrics=['mean_absolute_error', 'mean_squared_error'])

In [15]:
EPOCHS = 50
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.fit(
    train_x_new, train_y,
    epochs=EPOCHS, 
    validation_data=(test_x_new, test_y),
    batch_size=256, shuffle=True,
    callbacks = [early_stopping]
)

Train on 750156 samples, validate on 250053 samples
Epoch 1/50
750156/750156 [==============================] - 10s 13us/sample - loss: 1.4688 - mean_absolute_error: 0.9765 - mean_squared_error: 1.4688 - val_loss: 1.6022 - val_mean_absolute_error: 1.0627 - val_mean_squared_error: 1.6022
Epoch 2/50
750156/750156 [==============================] - 10s 13us/sample - loss: 1.4677 - mean_absolute_error: 0.9764 - mean_squared_error: 1.4677 - val_loss: 1.6746 - val_mean_absolute_error: 1.0002 - val_mean_squared_error: 1.6746
Epoch 3/50
750156/750156 [==============================] - 10s 13us/sample - loss: 1.4685 - mean_absolute_error: 0.9764 - mean_squared_error: 1.4685 - val_loss: 1.2600 - val_mean_absolute_error: 0.9355 - val_mean_squared_error: 1.2600
Epoch 4/50
750156/750156 [==============================] - 10s 13us/sample - loss: 1.4672 - mean_absolute_error: 0.9766 - mean_squared_error: 1.4672 - val_loss: 1.2645 - val_mean_absolute_error: 0.8782 - val_mean_squared_error: 1.2645
Epoc